In [ ]:
import pyodbc, pandas as pd, os, smtplib
from email.message import EmailMessage
import json


In [36]:
# Load secrets
with open('secrets.json') as f:
    secrets = json.load(f)

In [ ]:

conn_str = (
    f"DRIVER={{ODBC Driver 18 for SQL Server}};"
    f"SERVER={secrets['server']},1433;"
    f"DATABASE={secrets['database']};"
    f"UID={secrets['username']};"
    f"PWD={secrets['password']};"
    "Encrypt=no;"
    "TrustServerCertificate=yes;"
)

conn = pyodbc.connect(conn_str)

query1 = "SELECT * FROM dbo.OrderInformationQuery"
query2 = "SELECT * FROM dbo.ProductInformationQuery"


conn = pyodbc.connect(conn_str)
df1 = pd.read_sql(query1, conn)
df2 = pd.read_sql(query2, conn)
# print(df1.head())
# print("----------")
# print(df2.head())
os.makedirs("Output_CSV", exist_ok=True)
df1.to_csv(f"Output_CSV/OrderInformationQuery.csv", index=False)
df2.to_csv(f"Output_CSV/ProductInformationQuery.csv", index=False)


C:\Users\AlanC\AppData\Local\Temp\ipykernel_33152\2714422726.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df1 = pd.read_sql(query1, conn)
C:\Users\AlanC\AppData\Local\Temp\ipykernel_33152\2714422726.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df2 = pd.read_sql(query2, conn)


In [ ]:
# --- Email setup ---
email_sender = secrets['email_sender']
print(f"Email sender: {email_sender}")
email_sender_password = secrets['email_sender_password']

msg = EmailMessage()
msg["From"] = email_sender
msg["To"] = email_sender ### testing purposes, change to actual recipient
msg["Subject"] = "Daily SQL Report"
msg.set_content("Hi,\n\nPlease find attached today's report.\n\nBest regards.")

file_paths = ["Output_CSV/OrderInformationQuery.csv","Output_CSV/ProductInformationQuery.csv"]
# Attach the file
for file_path in file_paths:
    with open(file_path, "rb") as f:
        data = f.read()
        msg.add_attachment(data,maintype="text",subtype="csv",filename=os.path.basename(file_path))


# --- Send via SMTP ---
smtp = smtplib.SMTP("smtp.gmail.com", 587)    # change to your SMTP server(Gmail or Outlook)
smtp.ehlo()                                         
smtp.starttls()                                     
smtp.ehlo()                                         
smtp.login(email_sender, email_sender_password)               
smtp.send_message(msg)                           
smtp.quit()                                       


f.close()
conn.close()





Email sender: alanchen20072@gmail.com


(221,
 b'2.0.0 closing connection e9e14a558f8ab-3dd94ac99dcsm20887105ab.70 - gsmtp')

In [42]:
import shutil
shutil.rmtree("Output_CSV")